In [1]:
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\1506043\Downloads\AllData.csv")

In [ ]:
import requests
from io import StringIO
from datetime import datetime, timedelta

# Configuration
BASE_URL = 'https://www.emi.ea.govt.nz/Wholesale/Download/DataReport/CSV/CLA3WR'
NODE_CODES = ['OTA2201', 'WKM2201', 'RDF2201', 'HAY2201', 'KIK2201', 'ISL2201', 'BEN2201', 'INV2201']

# Date range: from 2013-07-01 to yesterday
start_date_str = '2013-07-01'
yesterday = datetime.now().date() - timedelta(days=1)
end_date_str = yesterday.strftime('%Y-%m-%d')

# Download spot price data for each node
data_frames = []

This may take a few minutes...


In [ ]:
with requests.Session() as session:
    for i, node_code in enumerate(NODE_CODES, 1):
        params = {
            'DateFrom': start_date_str,
            'DateTo': end_date_str,
            'POC': node_code,
            '_rsdr': 'W1',
            '_si': 'v|3'
        }
        
        try:
            response = session.get(BASE_URL, params=params)
            
            if response.status_code == 200:
                csv_file_in_memory = StringIO(response.text)
                node_df = pd.read_csv(csv_file_in_memory, skiprows=9)
                data_frames.append(node_df)
        
        except requests.RequestException as e:
            print(f"Error downloading {node_code}: {e}")

[1/8] Downloaded data for OTA2201
[2/8] Downloaded data for WKM2201
[3/8] Downloaded data for RDF2201
[4/8] Downloaded data for HAY2201
[5/8] Downloaded data for KIK2201
[6/8] Downloaded data for ISL2201
[7/8] Downloaded data for BEN2201
[8/8] Downloaded data for INV2201

Successfully downloaded data for 8 nodes


In [ ]:
# Merge and calculate daily averages
spot_data = pd.concat(data_frames, ignore_index=True)
spot_data['Trading date'] = pd.to_datetime(spot_data['Trading date'], dayfirst=True)

daily_avg_spot = (
    spot_data
    .groupby(['Trading date', 'Point of connection'])['$/MWh']
    .mean()
    .reset_index()
)

Spot price data shape: (34447, 3)

Date range: 2013-07-01 00:00:00 to 2025-11-13 00:00:00

Nodes: ['BEN2201', 'HAY2201', 'INV2201', 'ISL2201', 'KIK2201', 'OTA2201', 'RDF2201', 'WKM2201']

Sample data:
   Trading date Point of connection      $/MWh
0    2013-07-01             BEN2201  63.302708
1    2013-07-01             HAY2201  63.335000
2    2013-07-01             INV2201  62.448125
3    2013-07-01             ISL2201  67.149375
4    2013-07-01             OTA2201  65.544583
5    2013-07-01             RDF2201  63.294375
6    2013-07-01             WKM2201  64.058750
7    2013-07-02             BEN2201  55.539792
8    2013-07-02             HAY2201  58.030000
9    2013-07-02             INV2201  54.562917
10   2013-07-02             ISL2201  57.852083
11   2013-07-02             OTA2201  62.733958
12   2013-07-02             RDF2201  60.578125
13   2013-07-02             WKM2201  61.367500
14   2013-07-03             BEN2201  42.513333
15   2013-07-03             HAY2201  45.291875


In [ ]:
# Calculate monthly averages
daily_avg_spot['YYYYMM'] = daily_avg_spot['Trading date'].dt.strftime('%Y%m')

monthly_avg_spot = (
    daily_avg_spot
    .groupby(['YYYYMM', 'Point of connection'])['$/MWh']
    .mean()
    .reset_index()
)

Monthly average spot prices shape: (1136, 3)

Sample monthly averages:
    YYYYMM Point of connection      $/MWh
0   201307             BEN2201  48.807722
1   201307             HAY2201  53.113448
2   201307             INV2201  45.159751
3   201307             ISL2201  51.299758
4   201307             OTA2201  57.327493
5   201307             RDF2201  54.987460
6   201307             WKM2201  55.815141
7   201308             BEN2201  42.004523
8   201308             HAY2201  64.625961
9   201308             INV2201  40.449052
10  201308             ISL2201  43.915612
11  201308             OTA2201  67.116257
12  201308             RDF2201  64.463280
13  201308             WKM2201  65.529691
14  201309             BEN2201  51.410259
15  201309             HAY2201  62.114915
16  201309             INV2201  50.346805
17  201309             ISL2201  53.838263
18  201309             OTA2201  62.403346
19  201309             RDF2201  57.498164


In [ ]:
# Create obligation price differences (simple monthly averaging)
monthly_from = monthly_avg_spot.copy()
monthly_from.columns = ['YYYYMM', 'Node_From', 'Price_From']

monthly_to = monthly_avg_spot.copy()
monthly_to.columns = ['YYYYMM', 'Node_To', 'Price_To']

node_price_diffs = monthly_from.merge(monthly_to, on='YYYYMM')
node_price_diffs = node_price_diffs[node_price_diffs['Node_From'] != node_price_diffs['Node_To']]

# Calculate price difference: Price_To - Price_From (positive = receive money)
node_price_diffs['Price_Difference'] = node_price_diffs['Price_To'] - node_price_diffs['Price_From']

node_price_diffs['Source'] = node_price_diffs['Node_From'].str.replace('2201', '')
node_price_diffs['Sink'] = node_price_diffs['Node_To'].str.replace('2201', '')

obligation_price_diffs = node_price_diffs[['YYYYMM', 'Node_From', 'Node_To', 'Price_From', 'Price_To', 
                                             'Price_Difference', 'Source', 'Sink']].copy()

Node price differences shape: (7560, 6)

Sample price differences (e.g., OTA2201 to BEN2201):
     YYYYMM Node_From  Price_From  Node_To    Price_To  Price_Difference
24   201307   OTA2201   57.327493  BEN2201   48.807722          8.519772
66   201308   OTA2201   67.116257  BEN2201   42.004523         25.111734
108  201309   OTA2201   62.403346  BEN2201   51.410259         10.993087
150  201310   OTA2201   39.885128  BEN2201   22.160827         17.724301
192  201311   OTA2201   43.658153  BEN2201   22.286250         21.371903
234  201312   OTA2201   53.283790  BEN2201   39.267056         14.016734
276  201401   OTA2201   67.812816  BEN2201   54.228112         13.584704
318  201402   OTA2201  124.535134  BEN2201  110.979792         13.555342
360  201403   OTA2201  119.446519  BEN2201  117.710706          1.735813
402  201404   OTA2201   92.907962  BEN2201   95.361512         -2.453551


In [ ]:
# Create half-hourly price differences for options calculation
spot_data['YYYYMM'] = spot_data['Trading date'].dt.strftime('%Y%m')

spot_from = spot_data[['Trading date', 'Trading period', 'YYYYMM', 'Point of connection', '$/MWh']].copy()
spot_from.columns = ['Trading date', 'Trading period', 'YYYYMM', 'Node_From', 'Price_From']

spot_to = spot_data[['Trading date', 'Trading period', 'YYYYMM', 'Point of connection', '$/MWh']].copy()
spot_to.columns = ['Trading date', 'Trading period', 'YYYYMM', 'Node_To', 'Price_To']

half_hourly_diffs = spot_from.merge(spot_to, on=['Trading date', 'Trading period', 'YYYYMM'])
half_hourly_diffs = half_hourly_diffs[half_hourly_diffs['Node_From'] != half_hourly_diffs['Node_To']]

# Calculate price difference at half-hourly level
half_hourly_diffs['Price_Diff_HalfHour'] = half_hourly_diffs['Price_To'] - half_hourly_diffs['Price_From']

Creating half-hourly price differences for all node pairs...
This may take a moment...
Half-hourly differences calculated: 11,008,830 rows
Sample:
  Trading date  Trading period  YYYYMM Node_From  Price_From  Node_To  \
1   2025-11-13               1  202511   OTA2201       29.14  WKM2201   
2   2025-11-13               1  202511   OTA2201       29.14  RDF2201   
3   2025-11-13               1  202511   OTA2201       29.14  HAY2201   
4   2025-11-13               1  202511   OTA2201       29.14  KIK2201   
5   2025-11-13               1  202511   OTA2201       29.14  ISL2201   

   Price_To  Price_Diff_HalfHour  
1     27.54                 1.60  
2     27.43                 1.71  
3     25.34                 3.80  
4     26.00                 3.14  
5     25.71                 3.43  


In [ ]:
# Calculate option price differences (max(0, difference) at half-hourly, then average)
half_hourly_diffs['Price_Diff_Option'] = half_hourly_diffs['Price_Diff_HalfHour'].clip(lower=0)

option_price_diffs = (
    half_hourly_diffs
    .groupby(['YYYYMM', 'Node_From', 'Node_To'])
    .agg({
        'Price_From': 'mean',
        'Price_To': 'mean',
        'Price_Diff_Option': 'mean'
    })
    .reset_index()
)

option_price_diffs['Source'] = option_price_diffs['Node_From'].str.replace('2201', '')
option_price_diffs['Sink'] = option_price_diffs['Node_To'].str.replace('2201', '')
option_price_diffs.columns = ['YYYYMM', 'Node_From', 'Node_To', 'Avg_Price_From', 'Avg_Price_To', 
                               'Option_Price_Difference', 'Source', 'Sink']


=== OPTION PRICE DIFFERENCES TABLE ===
Table 'option_price_diffs' created with 7,560 rows
This table uses: max(0, Price_From - Price_To) averaged across all half-hours

Sample data:
   YYYYMM Node_From  Node_To  Avg_Price_From  Avg_Price_To  \
0  201307   BEN2201  HAY2201       48.807722     53.113448   
1  201307   BEN2201  INV2201       48.807722     45.159751   
2  201307   BEN2201  ISL2201       48.807722     51.299758   
3  201307   BEN2201  OTA2201       48.807722     57.327493   
4  201307   BEN2201  RDF2201       48.807722     54.987460   
5  201307   BEN2201  WKM2201       48.807722     55.815141   
6  201307   HAY2201  BEN2201       53.113448     48.807722   
7  201307   HAY2201  INV2201       53.113448     45.159751   
8  201307   HAY2201  ISL2201       53.113448     51.299758   
9  201307   HAY2201  OTA2201       53.113448     57.327493   

   Option_Price_Difference Source Sink  
0                 0.043239    BEN  HAY  
1                 3.653958    BEN  INV  
2          

In [ ]:
# Combine obligations and options into one table
obligation_table = obligation_price_diffs[['YYYYMM', 'Source', 'Sink', 'Price_Difference']].copy()
obligation_table['HedgeType'] = 'OBL'
obligation_table['key'] = obligation_table['YYYYMM'] + obligation_table['HedgeType'] + obligation_table['Source'] + obligation_table['Sink']
obligation_table = obligation_table[['key', 'Price_Difference']]
obligation_table.columns = ['key', 'price_difference']

option_table = option_price_diffs[['YYYYMM', 'Source', 'Sink', 'Option_Price_Difference']].copy()
option_table['HedgeType'] = 'OPT'
option_table['key'] = option_table['YYYYMM'] + option_table['HedgeType'] + option_table['Source'] + option_table['Sink']
option_table = option_table[['key', 'Option_Price_Difference']]
option_table.columns = ['key', 'price_difference']

combined_ftr_prices = pd.concat([obligation_table, option_table], ignore_index=True)
combined_ftr_prices = combined_ftr_prices.sort_values('key').reset_index(drop=True)

=== COMBINED FTR PRICE TABLE ===
Total rows: 15,120
Obligations: 7,560 rows
Options: 7,560 rows

Sample rows:
                key  price_difference
0   201307OBLBENHAY          4.305726
1   201307OBLBENINV         -3.647970
2   201307OBLBENISL          2.492036
3   201307OBLBENOTA          8.519772
4   201307OBLBENRDF          6.179738
5   201307OBLBENWKM          7.007419
6   201307OBLHAYBEN         -4.305726
7   201307OBLHAYINV         -7.953696
8   201307OBLHAYISL         -1.813690
9   201307OBLHAYOTA          4.214046
10  201307OBLHAYRDF          1.874012
11  201307OBLHAYWKM          2.701694
12  201307OBLINVBEN          3.647970
13  201307OBLINVHAY          7.953696
14  201307OBLINVISL          6.140007
15  201307OBLINVOTA         12.167742
16  201307OBLINVRDF          9.827708
17  201307OBLINVWKM         10.655390
18  201307OBLISLBEN         -2.492036
19  201307OBLISLHAY          1.813690


In [ ]:
# Join price differences to AllData
df_with_prices = df.merge(combined_ftr_prices, on='key', how='left')

JOINED DATA: AllData + Price Differences
Original df shape: (107235, 19)
Joined df shape: (107235, 20)

New columns: ['FTR_ID', 'AuctionName', 'StartDate', 'EndDate', 'CurrentOwner', 'PreviousOwner', 'FirstOwner', 'HedgeType', 'Source', 'Sink', 'MW', 'Price', 'AcquisitionCost', 'OriginalAcquisitionCost', 'DateAcquired', 'Status', 'AllocationPlan', 'key', 'YYYYMM', 'price_difference']

Sample joined data:
   FTR_ID HedgeType Source Sink  StartDate  Price              key  \
0   60152       OPT    OTA  BEN 2013-09-01    1.0  201309OPTOTABEN   
1   60148       OPT    BEN  OTA 2013-09-01    7.0  201309OPTBENOTA   
2   60146       OPT    OTA  BEN 2013-09-01    1.0  201309OPTOTABEN   
3   60142       OPT    OTA  BEN 2013-09-01    1.0  201309OPTOTABEN   
4   60141       OPT    BEN  OTA 2013-09-01    7.0  201309OPTBENOTA   
5   60133       OPT    OTA  BEN 2013-09-01    1.0  201309OPTOTABEN   
6   60128       OPT    OTA  BEN 2013-09-01    1.0  201309OPTOTABEN   
7   60121       OPT    BEN  OTA 

In [ ]:
# Calculate trade_profit
df_with_prices['StartDate'] = pd.to_datetime(df_with_prices['StartDate'])
df_with_prices['EndDate'] = pd.to_datetime(df_with_prices['EndDate'], format='%d/%m/%Y', errors='coerce')

df_with_prices['days'] = (df_with_prices['EndDate'] - df_with_prices['StartDate']).dt.days + 1

# For options: (max(0, price_difference) - Price) * MW * 24 * days
# For obligations: (price_difference - Price) * MW * 24 * days
df_with_prices['trade_profit'] = df_with_prices.apply(
    lambda row: (
        (max(0, row['price_difference']) - row['Price']) * row['MW'] * 24 * row['days']
        if row['HedgeType'] == 'OPT'
        else (row['price_difference'] - row['Price']) * row['MW'] * 24 * row['days']
    ) if pd.notna(row['price_difference']) else None,
    axis=1
)

TRADE PROFIT CALCULATION COMPLETE

Formula applied:
  OPT: (MAX(0, price_difference) - Price) × MW × 24 × days
  OBL: (price_difference - Price) × MW × 24 × days

Sample results:
  HedgeType Source Sink   MW  Price  price_difference  days  trade_profit
0       OPT    OTA  BEN  0.1    1.0          1.087643    30      6.310264
1       OPT    BEN  OTA  5.0    7.0         12.075668    30  18272.403338
2       OPT    OTA  BEN  0.1    1.0          1.087643    30      6.310264
3       OPT    OTA  BEN  4.8    1.0          1.087643    30    302.892684
4       OPT    BEN  OTA  4.5    7.0         12.075668    30  16445.163004
5       OPT    OTA  BEN  0.6    1.0          1.087643    30     37.861586
6       OPT    OTA  BEN  0.1    1.0          1.087643    30      6.310264
7       OPT    BEN  OTA  1.0    7.0         12.075668    30   3654.480668
8       OPT    BEN  OTA  5.0    7.0         12.075668    30  18272.403338
9       OPT    OTA  BEN  0.1    1.0          1.087643    30      6.310264


In [46]:
# Final dataset summary
print(f"Final dataset shape: {df_with_prices.shape}")
print(f"\nColumns: {df_with_prices.columns.tolist()}")
print(f"\nSample data:")
print(df_with_prices[['FTR_ID', 'HedgeType', 'Source', 'Sink', 'MW', 'Price', 'price_difference', 'trade_profit']].head(10))

# To save: df_with_prices.to_csv('AllData_with_prices.csv', index=False)

Final dataset shape: (107235, 22)

Columns: ['FTR_ID', 'AuctionName', 'StartDate', 'EndDate', 'CurrentOwner', 'PreviousOwner', 'FirstOwner', 'HedgeType', 'Source', 'Sink', 'MW', 'Price', 'AcquisitionCost', 'OriginalAcquisitionCost', 'DateAcquired', 'Status', 'AllocationPlan', 'key', 'YYYYMM', 'price_difference', 'days', 'trade_profit']

Sample data:
   FTR_ID HedgeType Source Sink   MW  Price  price_difference  trade_profit
0   60152       OPT    OTA  BEN  0.1    1.0          1.087643      6.310264
1   60148       OPT    BEN  OTA  5.0    7.0         12.075668  18272.403338
2   60146       OPT    OTA  BEN  0.1    1.0          1.087643      6.310264
3   60142       OPT    OTA  BEN  4.8    1.0          1.087643    302.892684
4   60141       OPT    BEN  OTA  4.5    7.0         12.075668  16445.163004
5   60133       OPT    OTA  BEN  0.6    1.0          1.087643     37.861586
6   60128       OPT    OTA  BEN  0.1    1.0          1.087643      6.310264
7   60121       OPT    BEN  OTA  1.0    

In [47]:
# Create VWAP table aggregated by CurrentOwner and key
vwap_table = df_with_prices.groupby(['CurrentOwner', 'key', 'StartDate', 'HedgeType', 'Source', 'Sink']).apply(
    lambda x: pd.Series({
        'VWAP': (x['Price'] * x['MW']).sum() / x['MW'].sum() if x['MW'].sum() > 0 else 0,
        'Total_MW': x['MW'].sum(),
        'Number_of_Trades': len(x),
        'Avg_Price': x['Price'].mean(),
        'Min_Price': x['Price'].min(),
        'Max_Price': x['Price'].max()
    })
).reset_index()

# Format StartDate for readability
vwap_table['YYYYMM'] = pd.to_datetime(vwap_table['StartDate']).dt.strftime('%Y-%m')
vwap_table['Hedge_Direction'] = vwap_table['Source'] + ' → ' + vwap_table['Sink']

# Reorder columns for better readability
vwap_table = vwap_table[[
    'CurrentOwner', 'YYYYMM', 'HedgeType', 'Hedge_Direction', 
    'Source', 'Sink', 'VWAP', 'Avg_Price', 'Total_MW', 
    'Number_of_Trades', 'Min_Price', 'Max_Price', 'key'
]].sort_values(['CurrentOwner', 'YYYYMM', 'HedgeType', 'Source', 'Sink'])

print(f"VWAP Table shape: {vwap_table.shape}")
print(f"\nSample VWAP data:")
print(vwap_table.head(20))
print(f"\nVWAP Statistics:")
print(vwap_table[['VWAP', 'Total_MW', 'Number_of_Trades']].describe())

VWAP Table shape: (28129, 13)

Sample VWAP data:
   CurrentOwner   YYYYMM HedgeType Hedge_Direction Source Sink       VWAP  \
0          AOEN  2023-04       OPT       BEN → HAY    BEN  HAY   7.110000   
1          AOEN  2023-08       OPT       BEN → HAY    BEN  HAY   9.000000   
2          AOEN  2023-08       OPT       HAY → RDF    HAY  RDF   4.000000   
3          AOEN  2023-09       OPT       BEN → HAY    BEN  HAY  12.000000   
4          AOEN  2023-09       OPT       BEN → KIK    BEN  KIK   0.000000   
5          AOEN  2023-09       OPT       HAY → BEN    HAY  BEN   0.400000   
6          AOEN  2023-09       OPT       WKM → RDF    WKM  RDF   0.320000   
7          AOEN  2023-10       OPT       BEN → HAY    BEN  HAY   7.800000   
8          AOEN  2023-10       OPT       BEN → ISL    BEN  ISL   0.000000   
9          AOEN  2023-10       OPT       BEN → KIK    BEN  KIK  11.680000   
10         AOEN  2023-10       OPT       INV → BEN    INV  BEN   1.832847   
11         AOEN  2023-11   

C:\Users\1506043\AppData\Local\Temp\ipykernel_13288\3417487067.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwap_table = df_with_prices.groupby(['CurrentOwner', 'key', 'StartDate', 'HedgeType', 'Source', 'Sink']).apply(


In [48]:
# Export both tables to Excel with two sheets
output_file = 'AllData_Analysis.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Export main table with all data
    df_with_prices.to_excel(writer, sheet_name='AllData', index=False)
    
    # Export VWAP table
    vwap_table.to_excel(writer, sheet_name='VWAP', index=False)

print(f"✓ Exported to: {output_file}")
print(f"  - Sheet 1: 'AllData' ({df_with_prices.shape[0]:,} rows × {df_with_prices.shape[1]} columns)")
print(f"  - Sheet 2: 'VWAP' ({vwap_table.shape[0]:,} rows × {vwap_table.shape[1]} columns)")

✓ Exported to: AllData_Analysis.xlsx
  - Sheet 1: 'AllData' (107,235 rows × 22 columns)
  - Sheet 2: 'VWAP' (28,129 rows × 13 columns)
